In [ ]:
%matplotlib inline
import os
from datetime import datetime
from descartes import PolygonPatch
import matplotlib as mpl
from matplotlib.collections import PatchCollection
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
from shapely import geometry as sg, wkt
from scripts.espm_module import CalAdaptRequest, GBIFRequest
plt.style.use('seaborn')

# ESPM / IB 105

The goal of this notebook is to access and integrate diverse data sets to visualize correlations and discover patterns to address questions of species’ responses to environmental change. We will use programmatic tools to show how to use Berkeley resources such as the biodiversity data from biocollections and online databases, field stations, climate models, and other environmental data.

Before we begin analyzing and visualizing biodiversity data, this introductory notebook will help familiarize you with the basics of programming in Python. This notebook will cover fundamental Python syntax, Pandas dataframes, and visualization using Matplotlib.

## Table of Contents

1 - [Introduction to Coding Concepts](#programming concepts)
    
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 1 - [Python Basics](#python basics)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 2 - [GBIF API](#pandas)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 3 - [Cal-Adapt API](#viz)

### Part 1: Python basics <a id='python basics'></a>
Before getting into the more advanced analysis techniques that will be required in this course, we need to cover a few of the foundational elements of programming in Python.
#### A. Expressions
The departure point for all programming is the concept of the __expression__. An expression is a combination of variables, operators, and other Python elements that the language interprets and acts upon. Expressions act as a set of instructions to be fed through the interpreter, with the goal of generating specific outcomes. See below for some examples of basic expressions.

In [ ]:
# Examples of expressions:

2 + 2

'me' + ' and I'

12 ** 2

6 + 4

You will notice that only the last line in a cell gets printed out. If you want to see the values of previous expressions, you need to call `print` on that expression. Try adding `print` statements to some of the above expressions to get them to display.

#### B. Variables
In the example below, `a` and `b` are Python objects known as __variables__. We are giving an object (in this case, an `integer` and a `float`, two Python data types) a name that we can store for later use. To use that value, we can simply type the name that we stored the value as. Variables are stored within the notebook's environment, meaning stored variable values carry over from cell to cell.

In [ ]:
a = 4
b = 10/5

Notice that when you create a variable, unlike what you previously saw with the expressions, it does not print anything out.

In [ ]:
# Notice that 'a' retains its value.
print(a)
a + b

#### C. Lists
The next topic is particularly useful in the kind of data manipulation that you will see throughout 101B. The following few cells will introduce the concept of __lists__ (and their counterpart, `numpy arrays`). Read through the following cell to understand the basic structure of a list. 

A list is an ordered collection of objects. They allow us to store and access groups of variables and other objects for easy access and analysis. Check out this [documentation](https://www.tutorialspoint.com/python/python_lists.htm) for an in-depth look at the capabilities of lists.

To initialize a list, you use brackets. Putting objects separated by commas in between the brackets will add them to the list. 

In [ ]:
# an empty list
lst = []
print(lst)

# reassigning our empty list to a new list
lst = [1, 3, 6, 'lists', 'are' 'fun', 4]
print(lst)

To access a value in the list, put the index of the item you wish to access in brackets following the variable that stores the list. Lists in Python are zero-indexed, so the indicies for `lst` are 0, 1, 2, 3, 4, 5, and 6.

In [ ]:
# Elements are selected like this:
example = lst[2]

# The above line selects the 3rd element of lst (list indices are 0-offset) and sets it to a variable named example.
print(example)

#### D. Functions

So far, you've learnt how to assign variables to certain values. Now, let's learn how we can manipulate some of these values.

Let's say we want to perform a certain operation on many different inputs that will produce distinct outputs. What do we do? We write a **_function_**.

A function is a block of code which works a lot like a machine: it takes an input, does something to it, and produces an output.

The input is put between brackets and can also be called the _argument_ or _parameter_. Functions can have multiple arguments.

Try running the cell below after changing the variable _name_:

In [ ]:
# Edit this cell to your own name!
name = "John Doe"

# Our function
def hello(name):
    return "Hello " + name + "!"

hello(name)

Interesting, right? Now, you don't need to write 10 different lines with 10 different names to print a special greeting for each person. All you need to is write one function that does all the work for you!

Now, let's write our own function. Let's look at the following rules:

**Defining**

* All functions must start with the "def" keyword.
* All functions must have a name, followed by parentheses, followed by a colon. Eg. def hello( ):
* The parentheses may have a variable that stores its arguments (inputs)
* All functions must have a "return" statement which will return the output. Think of a function like a machine. When you put something inside, you want it to return something. Hence, this is very important.

**Calling**

After you define a function, it's time to use it. This is known as _calling_ a function.

To call a function, simply write the name of the function with your input variable in brackets (argument).

In [ ]:
# Complete this function
def my_first_function(argument):
    return # function must return a value - REPLACE THIS LINE
           # have this function return the input

name = ... # FILL IN YOUR NAME HERE, in quotes

# Calling our function below...
my_first_function(name)

Great! Now let's do some math. Let's write a function that returns the square of the input.

Try writing it from scratch!

In [ ]:
# square function 
def square(num):
    return ...   # YOUR CODE HERE

# this should output 25!
# try inputting different numbers instead of 5 and see what happens
square(5)

Neat stuff! Try different inputs and check if you get the correct answer each time.

You've successfully written your first function from scratch!

---

# GBIF API

The Global Biodiversity Information Facility has created an API that we can use to get data about different species at the [GBIF Web API](http://www.gbif.org/developer/summary). It utilizes pagination to split up the returned JSON records, so we need to request each successive page to gather all the results.

Then, we can request the available occurrence records for a particular species and plot their locations. Here we're going to make a request for the species whose scientific name is 'Argia agrioides', also known as the [California dancer](https://www.google.com/search?q=Argia+agrioides&rlz=1C1CHBF_enUS734US734&source=lnms&tbm=isch&sa=X&ved=0ahUKEwji9t29kNTWAhVBymMKHWJ-ANcQ_AUICygC&biw=1536&bih=694). We're going to plot their locations and display other data results. 

In [ ]:
req = GBIFRequest()
params = {'scientificName': 'Argia agrioides'}
pages = req.get_pages(params)
# Filter all the returned records for valid locations.
records = [rec for page in pages for rec in page['results']
           if rec.get('decimalLatitude')]
# Make a list of coordinate pairs for plotting.
coords = [(r['decimalLongitude'], r['decimalLatitude']) for r in records]
xs, ys = zip(*coords)

We can then plot the species:

In [ ]:
# Make a plot of species' locations below.
plt.plot(xs, ys, 'ro', alpha=0.5)

For all the reserves, we will need to send a request to GeoJSON, which is a format for encoding a variety of geographic data structures. With this code, we can request GeoJSON for all reserves and plot ocurrences of the species.

In [ ]:
# This is the url where are data is located 
url = 'https://ecoengine.berkeley.edu/api/layers/reserves/features/'
# Make a request, which receives data from this url. Then iterate throuh each feature so we can collect data about them.
reserves = requests.get(url, params={'page_size': 30}).json()
geoms = {feat['properties']['name']: sg.asShape(feat['geometry']) for feat in reserves['features']}

In [ ]:
# Buffer the reserves so we can view them at a regional scale.
patches = [PolygonPatch(geom.buffer(0.25)) for geom in geoms.values()]
fig, ax = plt.subplots()
colors = 100 * np.random.rand(len(patches))
p = PatchCollection(patches, cmap='tab10', alpha=0.4)
p.set_array(np.array(colors))
ax.add_collection(p)
ax.autoscale()

Let's try and zoom in to the Santa Cruz Island Reserve where we have a single occurrence, and plot it similarly. 

In [ ]:
# Collect data about what we want to graph.
fig, ax = plt.subplots()
g = geoms['Santa Cruz Island Reserve']
poly = PolygonPatch(g)
ax.add_patch(poly)
# Plot a graph representing the Santa Cruz Island Reserve using the data we've accumulated in these variables.
ax.plot(xs, ys, 'ro', clip_path=poly)
xmin, ymin, xmax, ymax = g.bounds
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)

In the same way, we can parse the WKT geometry for Blodgett, Hopland, and Sagehen stations and plot them individually.

In [ ]:
station_urls = {
    'Blodgett': 'https://raw.githubusercontent.com/BNHM/spatial-layers/master/wkt/BlodgettForestResearchStation.wkt',
    'Hopland': 'https://raw.githubusercontent.com/BNHM/spatial-layers/master/wkt/HoplandREC.wkt',
    'Sagehen': 'https://raw.githubusercontent.com/BNHM/spatial-layers/master/wkt/SagehenCreekFieldStation.wkt'
}
stn_features = [{'id': name, 'geometry': wkt.loads(requests.get(url).text)}
                for name, url in station_urls.items()]
# Reduce geometric complexity, we need smaller geometry serializations to circumvent 
# URL length restrictions.
tol = .0001
for feat in stn_features:
    feat['geometry'] = feat['geometry'].buffer(tol).simplify(tol)

In [ ]:
f, axes = plt.subplots(1, len(stn_features))
f.set_size_inches((12, 6))

# Title our graphs.
for ax, feat in zip(axes, stn_features):
    ax.add_patch(PolygonPatch(feat['geometry']))
    ax.set_title(feat['id'])
    ax.autoscale()

This will use the [Cal-Adapt](http://api.cal-adapt.org/api/) web API to access a remote raster data source and preview a rendered image. In this case, the data source contains images of locations and temperature information corresponding to each one. 

In [ ]:
url = 'http://api.cal-adapt.org/api/series/tasmax_year_CanESM2_rcp85/rasters/'
json = requests.get(url).json()
tifurl = json['results'][0]['image']
title = json['results'][0]['slug']

In [ ]:
import greenwich

with greenwich.open(tifurl) as r:
    arr = r.masked_array()

Now we need to visualize the resulting masked array. The array values are maximum temperature in Kelvin.

In [ ]:
# Use Matplotlib to create a color-coded plot of the resulting masked array:
plt.imshow(arr, cmap='spectral')
plt.title(title)
plt.colorbar()

In [ ]:
# Use Matplotlib to plot a histogram (arr should be un-masked here)

h = plt.hist(arr.compressed(), 20)

---

# Cal-Adapt API

As we look closer at the Cal-Adapt API, the following will ease working with time series raster data. It will request an entire time series for any geometry and return a Pandas `DataFrame` object. This will make displaying the data much easier with Pandas' built in methods. 

In [ ]:
# Creating Pandas dataframe by combining requested time series with records_g
req = CalAdaptRequest()
records_g = [dict(rec, geometry=sg.Point(rec['decimalLongitude'], rec['decimalLatitude']))
             for rec in records]
df = req.concat_features(records_g, 'gbifID')

In [ ]:
df.iloc[:,:9].plot()
plt.title('Argia agrioides - %s' % req.slug)

We can map the projected means for occurrence locations.

In [ ]:
tmax_means = df.mean(axis=0)
# Filter GBIF records where we have climate data.
records_rvals = [rec for rec in records_g if rec['gbifID'] in tmax_means.index]
coords = [(r['decimalLongitude'], r['decimalLatitude']) for r in records_rvals]
xs2, ys2 = zip(*coords)
#plt.scatter(xs2, ys2, c=tmax_means, cmap='viridis', alpha=0.5)
norm = mpl.colors.Normalize()
size = np.pi * (15 * norm(tmax_means)) ** 2 
plt.scatter(xs2, ys2, c=tmax_means, s=size, cmap='viridis', alpha=0.5)
plt.colorbar()

In [ ]:
#plt.figure(figsize=(12, 6))
df.iloc[:,:7].plot.box()
plt.title('Argia agrioides - %s' % req.slug)

Taking a look at minimum temperatures now, here are the temperature distributions by decade.

In [ ]:
dfs = []
names = ('tasmin_year_CanESM2_rcp45', 'tasmin_year_CanESM2_rcp85')
for name in names:
    req = CalAdaptRequest(name)
    dfs.append(req.concat_features(records_g, 'gbifID'))

In [ ]:
for name, df in zip(names, dfs):
    t = df.resample('10A').mean().transpose()
    t.columns = t.columns.year
    t.plot.box()
    plt.title(name)

Make plots for historical and projected temperatures at three station locations. Once again, Pandas gives us a lot of functionality in displaying our data beautifully. 

In [ ]:
dfstns_lst = []
names = ('tasmax_year_CanESM2_historical', 'tasmax_year_CanESM2_rcp85')
# Iterate through each name and make requests using the CalAdapt API. We can also use Pandas to display this data efficiently.
for name in names:
    req = CalAdaptRequest(name)
    d = req.concat_features(stn_features)
    dfstns_lst.append(d)
    #d.plot()
    #plt.title(name)
dfstns = pd.concat(dfstns_lst)

In [ ]:
dfstns.describe()

In [ ]:
dfstns.plot()

In [ ]:
# Decadal average and difference from 20 years prior.
dfstns.resample('10A').mean().diff(periods=2).dropna()

In [ ]:
dfstns.resample('10A').mean().diff().plot()

Comparing California Oak species:
#1 Search on Quercus douglassi (Blue Oak), Quercus lobata (Valley Oak), Quercus durata (Leather Oak), Q. agrifolia (Coast Live Oak) within Bounding box of the field stations. 

Which field stations have which species? Rerun charting of localities on observed climate to answer: do they all experience the same yearly average maximums and minimums? Which species has the highest max tolerance? Minimum? How does that compare wtih the future climate models for those localities? Which species is predicted to experience the most change in the future?

#2 What are the specimens that occurs in Sagehen Creek FS (or any one of the three field stations)? (SCFS most likely to have checklist in Ecoengine...) 

Use: GBIF API, ecoengine api for checklists (also bigcb.berkeley.edu, although may not be available as web service???)
Search GBIF for specimens within the SCFS polygon. Create a unique species list.
Compare with the unique documented species list with the checklist species. Which species are expected but not documented? Are there any documented but not on the checklist?

What other meaningful spatial data can we intersect with species occurrence? With the field station polygons in order to compare how our field stations sample different habitats and ecoregions?